In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error

In [2]:
stocks=["^NSEI"]
ohclv_data={}
for ticker in stocks:
    temp=yf.download(ticker,period='5y',interval='1mo')
    temp.dropna(how='any',inplace=True)
    ohclv_data[ticker]=temp   

[*********************100%***********************]  1 of 1 completed


In [3]:
df=ohclv_data["^NSEI"]
# df=ohclv_data["SBIN.NS"]
df1=df["Close"]
df1

Date
2018-09-01    10930.450195
2018-10-01    10386.599609
2018-11-01    10876.750000
2018-12-01    10862.549805
2019-01-01    10830.950195
2019-02-01    10792.500000
2019-03-01    11623.900391
2019-04-01    11748.150391
2019-05-01    11922.799805
2019-06-01    11788.849609
2019-07-01    11118.000000
2019-08-01    11023.250000
2019-09-01    11474.450195
2019-10-01    11877.450195
2019-11-01    12056.049805
2019-12-01    12168.450195
2020-01-01    11962.099609
2020-02-01    11201.750000
2020-03-01     8597.750000
2020-04-01     9859.900391
2020-05-01     9580.299805
2020-06-01    10302.099609
2020-07-01    11073.450195
2020-08-01    11387.500000
2020-09-01    11247.549805
2020-10-01    11642.400391
2020-11-01    12968.950195
2020-12-01    13981.750000
2021-01-01    13634.599609
2021-02-01    14529.150391
2021-03-01    14690.700195
2021-04-01    14631.099609
2021-05-01    15582.799805
2021-06-01    15721.500000
2021-07-01    15763.049805
2021-08-01    17132.199219
2021-09-01    17618.150

## building the forecasting model using LSTM

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Load the stock market data
data = df[1000:] # Replace 'stock_data.csv' with your dataset

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Define the number of previous periods and future periods
num_periods = 20
future_periods = 4

# Prepare the data for training
train_data = scaled_data[:-num_periods]
X_train = []
y_train = []
for i in range(num_periods, len(train_data)):
    X_train.append(train_data[i - num_periods:i, 0])
    y_train.append(train_data[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Build and train the LSTM model
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(tf.keras.layers.LSTM(units=50,return_sequences=True))
model.add(tf.keras.layers.LSTM(units=50))
model.add(tf.keras.layers.Dense(units=1))
optim=tf.keras.optimizers.RMSprop(learning_rate=3e-4)
model.compile(optimizer=optim, loss='mean_squared_error')
history=model.fit(X_train, y_train, epochs=60, batch_size=32)

# Plot the loss curve
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Prepare the data for making future predictions
last_periods = scaled_data[-num_periods:]
X_future = np.array([last_periods])
X_future = np.reshape(X_future, (X_future.shape[0], X_future.shape[1], 1))

# Make predictions for the future periods
predicted_future = []
for _ in range(future_periods):
    prediction = model.predict(X_future)
    predicted_future.append(prediction[0, 0])
    X_future = np.concatenate((X_future[:, 1:, :], prediction.reshape(1, 1, 1)), axis=1)

predicted_future = np.array(predicted_future)
predicted_future = np.reshape(predicted_future, (predicted_future.shape[0], 1))
predicted_future = scaler.inverse_transform(predicted_future)

# Plot the historical and predicted data
historical_data = scaler.inverse_transform(scaled_data)
future_periods_index = np.arange(len(historical_data), len(historical_data) + future_periods)

plt.figure(figsize=(10, 6))
plt.plot(historical_data, label='Historical Data')
plt.plot(future_periods_index, predicted_future, label='Predicted Future')
plt.title('Stock Market Predictions')
plt.xlabel('Period')
plt.ylabel('Closing Price')
plt.legend()
plt.show()

In [ ]:
predicted_future


In [ ]:
# Plot the historical and predicted data
historical_data = scaler.inverse_transform(scaled_data)
future_periods_index = np.arange(len(historical_data), len(historical_data) + future_periods)

plt.figure(figsize=(10, 6))
plt.plot(historical_data, label='Historical Data')
plt.plot(future_periods_index, predicted_future, label='Predicted Future')
plt.title('Stock Market Predictions')
plt.xlabel('Period')
plt.ylabel('Closing Price')
plt.legend()
plt.show()